## Imports

In [268]:
import requests
import time
import pandas as pd

## Getting Request

In [269]:
url = 'https://www.reddit.com/r/upliftingnews/.json'

#create custom user agent

headers = {'User-agent': 'JavierM'}

res= requests.get(url, headers=headers)

#4 or 5 error is bad
res.status_code

200

In [270]:
res.content

data = res.json()

## Looking Around The Json

In [271]:
sorted(data.keys())

['data', 'kind']

In [272]:
data['kind']

'Listing'

In [273]:
#everything we care about is in data

sorted(data['data'].keys())

['after', 'before', 'children', 'dist', 'modhash']

In [274]:
#posts are located in children
len(data['data']['children'])

25

In [275]:
data['data']['children'][0]['data']

{'approved_at_utc': None,
 'subreddit': 'UpliftingNews',
 'selftext': '',
 'author_fullname': 't2_15huce2v',
 'saved': False,
 'mod_reason_title': None,
 'gilded': 0,
 'clicked': False,
 'title': 'Gently stroking a baby reduces activity in their brain associated with painful experiences, a study has found',
 'link_flair_richtext': [],
 'subreddit_name_prefixed': 'r/UpliftingNews',
 'hidden': False,
 'pwls': 6,
 'link_flair_css_class': None,
 'downs': 0,
 'thumbnail_height': 78,
 'hide_score': False,
 'name': 't3_a7c9bt',
 'quarantine': False,
 'link_flair_text_color': 'dark',
 'author_flair_background_color': '',
 'subreddit_type': 'public',
 'ups': 110,
 'domain': 'bbc.co.uk',
 'media_embed': {},
 'thumbnail_width': 140,
 'author_flair_template_id': None,
 'is_original_content': False,
 'user_reports': [],
 'secure_media': None,
 'is_reddit_media_domain': False,
 'is_meta': False,
 'category': None,
 'secure_media_embed': {},
 'link_flair_text': None,
 'can_mod_post': False,
 'score':

In [276]:
i=1

In [277]:
data['data']['children'][i]['data']['title']

'Burnout, stress lead more companies to try a four-day work week. It leads to higher productivity, more motivated staff.'

In [278]:
titles = []
subreddits = []
upvotes = []
comment_slugs = []

In [279]:
titles.append(data['data']['children'][i]['data']['title'])
subreddits.append(data['data']['children'][i]['data']['subreddit'])
upvotes.append(data['data']['children'][i]['data']['ups'])
comment_slugs.append(data['data']['children'][i]['data']['permalink'].split('comments/')[1])

In [280]:
df = pd.DataFrame(columns= ['title','subreddit','upvotes','slugs'])

In [281]:
df['title'] = titles
df['subreddit'] = subreddits
df['upvotes'] = upvotes
df['slugs'] = comment_slugs

In [15]:
df

,title,subreddit,upvotes,slugs
0,Singaporean Gay Man Wins Right to Adopt Child ...,UpliftingNews,311,a76dcc/singaporean_gay_man_wins_right_to_adopt...


In [16]:
#id of the last post in our list
data['data']['after']

't3_a765pa'

In [17]:
[post['data']['name'] for post in data['data']['children']]

['t3_a70nsa',
 't3_a76dcc',
 't3_a7c9bt',
 't3_a7df7l',
 't3_a7ayyl',
 't3_a78y3z',
 't3_a7b0f3',
 't3_a7b6gk',
 't3_a78bmy',
 't3_a7d2nh',
 't3_a7btzu',
 't3_a7c9ak',
 't3_a79ix5',
 't3_a7cw9c',
 't3_a7b28l',
 't3_a777tc',
 't3_a7945i',
 't3_a709sf',
 't3_a6sq8r',
 't3_a7b7ov',
 't3_a7d4j4',
 't3_a7d47f',
 't3_a75okb',
 't3_a7csn4',
 't3_a765pa']

In [18]:
#this needs to be the anchor for the next time you hit reddits api
#t3_a68jky

In [19]:
#param = {'after': 't3_a68jky'}

In [20]:
#requests.get(url,params=param, headers=headers)

## Creating a loop to gather posts

In [21]:
url2 = 'https://reddit.com/r/upliftingnews/comments/'+ data['data']['children'][i]['data']['permalink'].split('comments/')[1]+'.json'
res2 = requests.get(url2, headers=headers)

In [22]:
url2

'https://reddit.com/r/upliftingnews/comments/a76dcc/singaporean_gay_man_wins_right_to_adopt_child_in/.json'

In [23]:
res2.status_code

200

In [24]:
commentdata = res2.json()

In [25]:
commentdata[0]

{'kind': 'Listing',
 'data': {'modhash': '',
  'dist': 1,
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'UpliftingNews',
     'selftext': '',
     'user_reports': [],
     'saved': False,
     'mod_reason_title': None,
     'gilded': 0,
     'clicked': False,
     'title': 'Singaporean Gay Man Wins Right to Adopt Child in Landmark Ruling',
     'link_flair_richtext': [],
     'subreddit_name_prefixed': 'r/UpliftingNews',
     'hidden': False,
     'pwls': 6,
     'link_flair_css_class': None,
     'downs': 0,
     'thumbnail_height': 78,
     'parent_whitelist_status': 'all_ads',
     'hide_score': False,
     'name': 't3_a76dcc',
     'quarantine': False,
     'link_flair_text_color': 'dark',
     'upvote_ratio': 0.9,
     'author_flair_background_color': None,
     'subreddit_type': 'public',
     'ups': 311,
     'domain': 'advocate.com',
     'media_embed': {},
     'thumbnail_width': 140,
     'author_flair_template_id': None,
     'is_origi

In [26]:
commentdata[1]['data']['children'][42]

IndexError: list index out of range

In [27]:
def get_data_from_subreddits(subreddit, numberofpages = 40, top=None):    
    titles = []
    subreddits = []
    upvotes = []
    comment_slugs = []
    comments = []
    number_of_comments = []
    df = pd.DataFrame(columns= ['title','subreddit','upvotes','slugs'])
    after = None
    headers = {'User-agent': 'JavierM'}
    
    if top == None:
        url = 'https://www.reddit.com/r/'+subreddit
    elif top != None:
        url = 'https://www.reddit.com/r/'+subreddit+'/top/?t='+top
    else:
        url= 0
        
    for x in range(numberofpages):
        print('Looping through page number ', x+1)
        if after == None:
            params = {}
        else:
            params = {'after': after}
        url1 = url+'/.json'
        print(url1)
        res = requests.get(url1, params=params, headers=headers)
        if res.status_code == 200:
            data = res.json()
            for i in range(len(data['data']['children'])):
                subreddit_comments = []
                titles.append(data['data']['children'][i]['data']['title'])
                subreddits.append(data['data']['children'][i]['data']['subreddit'])
                upvotes.append(data['data']['children'][i]['data']['ups'])
                comment_slugs.append(data['data']['children'][i]['data']['permalink'].split('comments/')[1])
                time.sleep(0.25)
                url2 = 'https://reddit.com/r/'+subreddit+'/comments/'+ data['data']['children'][i]['data']['permalink'].split('comments/')[1]+'.json'
                res2 = requests.get(url2, headers=headers)
                if res2.status_code == 200:
                    print('Scanning post ' + str(i+1))
                    commentdata = res2.json() 
                    for c in range(len(commentdata[1]['data']['children'])-1):
                        comment = commentdata[1]['data']['children'][c]['data']['body']
                        subreddit_comments.append(comment)
                        time.sleep(.05)
                    number_of_comments.append(len(commentdata[1]['data']['children']))
                else:
                    print(res2.status_code)
                    break
                comments.append(subreddit_comments)
            after = data['data']['after']
        else:
            print(res.status_code)
            break
        time.sleep(1) # input this to not hit the servers so hard
    print('DONE!')
    df['title'] = titles
    df['subreddit'] = subreddits
    df['upvotes'] = upvotes
    df['slugs'] = comment_slugs
    df['comments'] = comments
    df['number_of_comments'] = number_of_comments
    return df

In [29]:
df = get_data_from_subreddits('upliftingnews', numberofpages=40, top=None)

Looping through page number  1
https://www.reddit.com/r/upliftingnews/.json
Scanning post 1
Scanning post 2
Scanning post 3
Scanning post 4
Scanning post 5
Scanning post 6
Scanning post 7
Scanning post 8
Scanning post 9
Scanning post 10
Scanning post 11
Scanning post 12
Scanning post 13
Scanning post 14
Scanning post 15
Scanning post 16
Scanning post 17
Scanning post 18
Scanning post 19
Scanning post 20
Scanning post 21
Scanning post 22
Scanning post 23
Scanning post 24
Scanning post 25
Looping through page number  2
https://www.reddit.com/r/upliftingnews/.json
Scanning post 1
Scanning post 2
Scanning post 3
Scanning post 4
Scanning post 5
Scanning post 6
Scanning post 7
Scanning post 8
Scanning post 9
Scanning post 10
Scanning post 11
Scanning post 12
Scanning post 13
Scanning post 14
Scanning post 15
Scanning post 16
Scanning post 17
Scanning post 18
Scanning post 19
Scanning post 20
Scanning post 21
Scanning post 22
Scanning post 23
Scanning post 24
Scanning post 25
Looping through 

Scanning post 16
Scanning post 17
Scanning post 18
Scanning post 19
Scanning post 20
Scanning post 21
Scanning post 22
Scanning post 23
Scanning post 24
Scanning post 25
Looping through page number  18
https://www.reddit.com/r/upliftingnews/.json
Scanning post 1
Scanning post 2
Scanning post 3
Scanning post 4
Scanning post 5
Scanning post 6
Scanning post 7
Scanning post 8
Scanning post 9
Scanning post 10
Scanning post 11
Scanning post 12
Scanning post 13
Scanning post 14
Scanning post 15
Scanning post 16
Scanning post 17
Scanning post 18
Scanning post 19
Scanning post 20
Scanning post 21
Scanning post 22
Scanning post 23
Scanning post 24
Scanning post 25
Looping through page number  19
https://www.reddit.com/r/upliftingnews/.json
Scanning post 1
Scanning post 2
Scanning post 3
Scanning post 4
Scanning post 5
Scanning post 6
Scanning post 7
Scanning post 8
Scanning post 9
Scanning post 10
Scanning post 11
Scanning post 12
Scanning post 13
Scanning post 14
Scanning post 15
Scanning post 

Scanning post 19
Scanning post 20
Scanning post 21
Scanning post 22
Scanning post 23
Scanning post 24
Scanning post 25
Looping through page number  35
https://www.reddit.com/r/upliftingnews/.json
Scanning post 1
Scanning post 2
Scanning post 3
Scanning post 4
Scanning post 5
Scanning post 6
Scanning post 7
Scanning post 8
Scanning post 9
Scanning post 10
Scanning post 11
Scanning post 12
Scanning post 13
Scanning post 14
Scanning post 15
Scanning post 16
Scanning post 17
Scanning post 18
Scanning post 19
Scanning post 20
Scanning post 21
Scanning post 22
Scanning post 23
Scanning post 24
Scanning post 25
Looping through page number  36
https://www.reddit.com/r/upliftingnews/.json
Scanning post 1
Scanning post 2
Scanning post 3
Scanning post 4
Scanning post 5
Scanning post 6
Scanning post 7
Scanning post 8
Scanning post 9
Scanning post 10
Scanning post 11
Scanning post 12
Scanning post 13
Scanning post 14
Scanning post 15
Scanning post 16
Scanning post 17
Scanning post 18
Scanning post 

In [30]:
df

,title,subreddit,upvotes,slugs,comments,number_of_comments
0,"Burnout, stress lead more companies to try a f...",UpliftingNews,60519,a70nsa/burnout_stress_lead_more_companies_to_t...,[I've been working this kind of schedule for t...,26
1,Singaporean Gay Man Wins Right to Adopt Child ...,UpliftingNews,317,a76dcc/singaporean_gay_man_wins_right_to_adopt...,[Just a reminder [Section 377A of the Penal Co...,2
2,Gently stroking a baby reduces activity in the...,UpliftingNews,25,a7c9bt/gently_stroking_a_baby_reduces_activity...,"[Pretty sure this works for adults too., Every...",3
3,Officer pens open letter to 18-year-old he sto...,UpliftingNews,16,a7df7l/officer_pens_open_letter_to_18yearold_h...,"[I’m not clicking on a yahoo link. , As a pare...",5
4,From no cure to no trace | Central Texas girl'...,UpliftingNews,22,a7ayyl/from_no_cure_to_no_trace_central_texas_...,[The last parents who cured there child's inop...,2
5,Scientists are working on a 90% efficient sola...,UpliftingNews,60,a78y3z/scientists_are_working_on_a_90_efficien...,"[Where I live, we average about 70% cloud cove...",6
6,The owner of the company my mom works for is g...,UpliftingNews,19,a7b0f3/the_owner_of_the_company_my_mom_works_f...,[],0
7,India opens a hospital for elephants,UpliftingNews,9,a7d2nh/india_opens_a_hospital_for_elephants/,[],0
8,"Girl's inoperable brain tumor disappears, doct...",UpliftingNews,6,a7e6ye/girls_inoperable_brain_tumor_disappears...,"[hallelujah, thank you JEESUS!]",2
9,Israeli lab reveals world's first lab-grown steak,UpliftingNews,16,a7b6gk/israeli_lab_reveals_worlds_first_labgro...,"[Unworthy thought, I know, but my head finishe...",2


In [31]:
df.to_csv('Uplifting_Tuesday_121818_2PM')

In [32]:
titles = []
subreddits = []
upvotes = []
comment_slugs = []
comments = []
number_of_comments = []
df = pd.DataFrame(columns= ['title','subreddit','upvotes','slugs'])
after = None
headers = {'User-agent': 'JavierM'}
for x in range(40):
    print('Looping through page number ', x+1)
    if after == None:
        params = {}
    else:
        params = {'after': after}
    url = 'https://www.reddit.com/r/news/.json'
    res = requests.get(url, params=params, headers=headers)
    if res.status_code == 200:
        data = res.json()
        for i in range(len(data['data']['children'])):
            subreddit_comments = []
            titles.append(data['data']['children'][i]['data']['title'])
            subreddits.append(data['data']['children'][i]['data']['subreddit'])
            upvotes.append(data['data']['children'][i]['data']['ups'])
            comment_slugs.append(data['data']['children'][i]['data']['permalink'].split('comments/')[1])
            time.sleep(0.25)
            url2 = 'https://reddit.com/r/news/comments/'+ data['data']['children'][i]['data']['permalink'].split('comments/')[1]+'.json'
            res2 = requests.get(url2, headers=headers)
            if res2.status_code == 200:
                print('Scanning post ' + str(i+1))
                commentdata = res2.json()
                
                for c in range(len(commentdata[1]['data']['children'])-1):
                    comment = commentdata[1]['data']['children'][c]['data']['body']
                    subreddit_comments.append(comment)
                    time.sleep(.05)
                number_of_comments.append(len(commentdata[1]['data']['children']))
            else:
                print(res2.status_code)
                break
            comments.append(subreddit_comments)
        after = data['data']['after']
    else:
        print(res.status_code)
        break
    time.sleep(1) # input this to not hit the servers so hard
print('DONE!')
df['title'] = titles
df['subreddit'] = subreddits
df['upvotes'] = upvotes
df['slugs'] = comment_slugs
df['comments'] = comments
df['number_of_comments'] = number_of_comments

Looping through page number  1
Scanning post 1
Scanning post 2
Scanning post 3
Scanning post 4
Scanning post 5
Scanning post 6
Scanning post 7
Scanning post 8
Scanning post 9
Scanning post 10
Scanning post 11
Scanning post 12
Scanning post 13
Scanning post 14
Scanning post 15
Scanning post 16
Scanning post 17
Scanning post 18
Scanning post 19
Scanning post 20
Scanning post 21
Scanning post 22
Scanning post 23
Scanning post 24
Scanning post 25
Looping through page number  2
Scanning post 1
Scanning post 2
Scanning post 3
Scanning post 4
Scanning post 5
Scanning post 6
Scanning post 7
Scanning post 8
Scanning post 9
Scanning post 10
Scanning post 11
Scanning post 12
Scanning post 13
Scanning post 14
Scanning post 15
Scanning post 16
Scanning post 17
Scanning post 18
Scanning post 19
Scanning post 20
Scanning post 21
Scanning post 22
Scanning post 23
Scanning post 24
Scanning post 25
Looping through page number  3
Scanning post 1
Scanning post 2
Scanning post 3
Scanning post 4
Scanning po

Scanning post 9
Scanning post 10
Scanning post 11
Scanning post 12
Scanning post 13
Scanning post 14
Scanning post 15
Scanning post 16
Scanning post 17
Scanning post 18
Scanning post 19
Scanning post 20
Scanning post 21
Scanning post 22
Scanning post 23
Scanning post 24
Scanning post 25
Looping through page number  21
Scanning post 1
Scanning post 2
Scanning post 3
Scanning post 4
Scanning post 5
Scanning post 6
Scanning post 7
Scanning post 8
Scanning post 9
Scanning post 10
Scanning post 11
Scanning post 12
Scanning post 13
Scanning post 14
Scanning post 15
Scanning post 16
Scanning post 17
Scanning post 18
Scanning post 19
Scanning post 20
Scanning post 21
Scanning post 22
Scanning post 23
Scanning post 24
Scanning post 25
Looping through page number  22
Scanning post 1
Scanning post 2
Scanning post 3
Scanning post 4
Scanning post 5
Scanning post 6
Scanning post 7
Scanning post 8
Scanning post 9
Scanning post 10
Scanning post 11
Scanning post 12
Scanning post 13
Scanning post 14
Sca

Scanning post 23
Scanning post 24
Scanning post 25
Looping through page number  39
Scanning post 1
Scanning post 2
Scanning post 3
Scanning post 4
Scanning post 5
Scanning post 6
Scanning post 7
Scanning post 8
Scanning post 9
Scanning post 10
Scanning post 11
Scanning post 12
Scanning post 13
Scanning post 14
Scanning post 15
Scanning post 16
Scanning post 17
Scanning post 18
Scanning post 19
Scanning post 20
Scanning post 21
Scanning post 22
Scanning post 23
Scanning post 24
Scanning post 25
Looping through page number  40
Scanning post 1
Scanning post 2
Scanning post 3
Scanning post 4
Scanning post 5
Scanning post 6
Scanning post 7
Scanning post 8
Scanning post 9
Scanning post 10
Scanning post 11
Scanning post 12
Scanning post 13
Scanning post 14
Scanning post 15
Scanning post 16
Scanning post 17
Scanning post 18
Scanning post 19
Scanning post 20
Scanning post 21
Scanning post 22
Scanning post 23
Scanning post 24
Scanning post 25
DONE!


In [33]:
df

,title,subreddit,upvotes,slugs,comments,number_of_comments
0,Trump Foundation agrees to dissolve under cour...,news,33174,a7cddu/trump_foundation_agrees_to_dissolve_und...,[From [forbes](https://www.forbes.com/sites/da...,31
1,Fortnite sued by Fresh Prince star Alfonso Rib...,news,20819,a79ru4/fortnite_sued_by_fresh_prince_star_alfo...,"[When he was on Dancing with the Stars, my dau...",33
2,Trump administration officially bans bump stocks,news,771,a7cm4w/trump_administration_officially_bans_bu...,[Except the way they did it is likely to bring...,30
3,Federal court says NY ban on nunchucks unconst...,news,29282,a76obd/federal_court_says_ny_ban_on_nunchucks/,[This is a great day for ninjas in this countr...,32
4,Human rights body calls on US school to ban el...,news,1047,a7a9dc/human_rights_body_calls_on_us_school_to...,"[Who is sending their kid to this school..., &...",43
5,The Latest: Flynn tells judge he knew lying to...,news,257,a7cw9b/the_latest_flynn_tells_judge_he_knew_ly...,[The guy was a general. Of course he knew that...,10
6,Second Woman Says Jeffrey Epstein ‘Directed’ H...,news,315,a7cbdz/second_woman_says_jeffrey_epstein_direc...,"[two POS pedos, one still being invited to fla...",12
7,Man shoots a bald eagle in Upstate NY. Said it...,news,239,a7cyl5/man_shoots_a_bald_eagle_in_upstate_ny_s...,"[Unless you've got animals (pigeons, rabbits, ...",15
8,Mother stabs rapist to death after catching hi...,news,69057,a73a1f/mother_stabs_rapist_to_death_after_catc...,[&gt;Each man was sentenced to 30 years in pri...,51
9,112-year-old veteran Richard Overton in the ho...,news,355,a7apo7/112yearold_veteran_richard_overton_in_the/,"[This guy lives down the street from me, he us...",10


In [34]:
df.to_csv('News_Tuesday_121818_240PM')

In [74]:
df1 = pd.read_csv('./Uplifting_Sunday_121618_1PM')
df2 = pd.read_csv('./Uplifting_Sunday_121718_930AM')
df3 = pd.read_csv('./News_Sunday_121618_130PM')
df4 = pd.read_csv('./News_Sunday_121718_10AM')

In [79]:
df = pd.concat([df1,df2,df3,df4],ignore_index=True)

In [84]:
df = df.iloc[:,1:]

In [86]:
df['title'].value_counts()

Alabama elementary school bus driver buys McDonald's breakfast for students when ice delays school opening                                                                                                                                                                                                13
Thieves stole from a stockpile of toys meant for thousands of local children in need, but a couple stepped in to save Christmas, raising $45,000 to buy new gifts for the kids.                                                                                                                           13
Workers in the video games industry have formed a union for the first time in the UK.                                                                                                                                                                                                                     13
Star Economist at Harvard Faces Sexual Harassment Complaints                                     

,title,subreddit,upvotes,slugs,comments,number_of_comments
0,Workers in the video games industry have forme...,UpliftingNews,9657,a6mwkn/workers_in_the_video_games_industry_hav...,['I am 1000% ok with waiting longer for games ...,50
1,Ruth Bader Ginsburg says she's 'almost repaire...,UpliftingNews,3047,a6ndpb/ruth_bader_ginsburg_says_shes_almost_re...,"['Odd choice of words', '“Almost repaired... I...",54
2,Negotiators in Poland have finally secured agr...,UpliftingNews,219,a6ny6a/negotiators_in_poland_have_finally_secu...,[],1
3,Cheers To Corona—It's The First Major Beer Bra...,UpliftingNews,43212,a6fsas/cheers_to_coronaits_the_first_major_bee...,"[""Why don't they just use cardboard boxes?"", '...",41
4,ISIS has lost its last stronghold. US backed a...,UpliftingNews,5622,a6ek5y/isis_has_lost_its_last_stronghold_us_ba...,"['# *ding, dong, DAESH is dead...*', ""as i sai...",43
5,Husky trapped on roof thanks the Firefighter W...,UpliftingNews,128,a6p4iu/husky_trapped_on_roof_thanks_the_firefi...,"[""Why didn't the owner just come home and open...",2
6,Trucker saves dogs thrown from vehicle on New ...,UpliftingNews,40,a6nejf/trucker_saves_dogs_thrown_from_vehicle_...,[],1
7,"Man donates $10,000 Christmas shopping spree f...",UpliftingNews,6,a6r2vu/man_donates_10000_christmas_shopping_sp...,[],0
8,An Indian surgeon becomes world's first to per...,UpliftingNews,13,a6ovxj/an_indian_surgeon_becomes_worlds_first_...,[],1
9,Tattoo shop owner offers credit towards free t...,UpliftingNews,1313,a6ep52/tattoo_shop_owner_offers_credit_towards...,"['I would love for a shop to do that here. ', ...",7


In [291]:
def get_data_from_pushshift(subreddit, epoch, numberofposts = 1000):    
    titles = []
    subreddits = []
    score = []
    ids = []
    df = pd.DataFrame(columns= ['title','subreddit','score','id'])
    headers = {'User-agent': 'JavierM'}
    url = 'https://api.pushshift.io/reddit/search/submission/'
    params = {
        'subreddit':subreddit,
        'after':epoch,
        'score':'>10',
        'size': numberofposts

    }

    res = requests.get(url, params=params, headers=headers)
    if res.status_code == 200:
        data = res.json()
        print(len(data['data']))
        for i in range(len(data['data'])):
            titles.append(data['data'][i]['title'])
            subreddits.append(data['data'][i]['subreddit'])
            score.append(data['data'][i]['score'])
            ids.append(data['data'][i]['id'])
    else:
        print(res.status_code)
    time.sleep(1) # input this to not hit the servers so hard
    print('DONE!')
    df['title'] = titles
    df['subreddit'] = subreddits
    df['score'] = score
    df['id'] = ids
    return df


In [292]:
get_data_from_pushshift('upliftingnews','1522559189')

500
DONE!


,title,subreddit,score,id
0,'Easter miracle': Aussie three-year-old missin...,UpliftingNews,95,88phgk
1,A team from Japanese research institutes wants...,UpliftingNews,26,88qcmz
2,Scotland produces record amount of energy from...,UpliftingNews,472,88qu1p
3,"Bullied boy with two colored eyes, cleft lip r...",UpliftingNews,25277,88re3i
4,A $29 Million Cryptocurrency Donation Just Fun...,UpliftingNews,30,88soog
5,Afghan woman who took exam with baby on lap en...,UpliftingNews,32,88srll
6,Parents turn life support off but 2 year old c...,UpliftingNews,451,88u8sl
7,The BBC aims to have an equal number of male a...,UpliftingNews,12,88uwbl
8,As the number of foreign children attending Ja...,UpliftingNews,39,88v26r
9,Man drops 326 pounds after needing two plane s...,UpliftingNews,32597,88xbhd


In [ ]:
subreddits ['upliftingnews','news']

In [247]:
url = 'https://api.pushshift.io/reddit/search/submission/'

#create custom user agent

headers = {'User-agent': 'JavierM'}
params = {
    'subreddit':'upliftingnews',
    'after':'1522559189',
    'score':'>10',
    'size': 500
    
}
res= requests.get(url, headers=headers, params=params)

#4 or 5 error is bad
res.status_code

200

In [250]:
data = res.json()

In [251]:
len(data['data'])

500

In [214]:
data['data'][0]['title']

"'Easter miracle': Aussie three-year-old missing for 18 hours found"

In [192]:
url = 'https://api.pushshift.io/reddit/search/comment/'

#create custom user agent

headers = {'User-agent': 'JavierM'}
params = {
    'ids':88phgk
    
}
res= requests.get(url, headers=headers, params=params)

#4 or 5 error is bad
res.status_code

SyntaxError: invalid syntax (<ipython-input-192-06da8c3af3f5>, line 7)

In [190]:
res.content

data = res.json()

In [191]:
data

{'data': []}